In [5]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

LOG_FILE = "../grid_search_result/backend_deployment_logs_FIXED.csv"

def analyze_realistic_performance():
    if not os.path.exists(LOG_FILE):
        print("❌ 파일 없음")
        return

    print(">>> 현실적(Non-Overlapping) 성과 분석 중...")
    df = pd.read_csv(LOG_FILE)
    
    # 시간 정렬 필수
    df['signal_time'] = pd.to_datetime(df['signal_time'])
    df = df.sort_values(['strategy', 'model', 'horizon_hours', 'cusum_multiplier', 'signal_time'])
    
    # 그룹별로 분석
    grouped = df.groupby(['strategy', 'model', 'horizon_hours', 'cusum_multiplier'])
    
    realistic_results = []
    
    for name, group in tqdm(grouped):
        strat, model, horizon, cusum = name
        
        # 보유 기간 (시간)
        hold_hours = int(horizon)
        
        # 시뮬레이션 변수
        trades = 0
        wins = 0
        last_exit_time = pd.Timestamp.min # 마지막 청산 시간
        
        # 루프 돌면서 중복 제거
        for _, row in group.iterrows():
            # 모델이 매수 신호를 보냈고, 현재 포지션이 없을 때만 진입
            if row['ml_prediction'] == 1 and row['signal_time'] >= last_exit_time:
                trades += 1
                if row['correct']:
                    wins += 1
                
                # 진입했으므로 종료 시간 설정 (이 시간 전까지는 진입 금지)
                last_exit_time = row['signal_time'] + pd.Timedelta(hours=hold_hours)
        
        if trades > 0:
            win_rate = wins / trades
            realistic_results.append({
                'strategy_name': strat,
                'model': model,
                'horizon_hours': horizon,
                'cusum_multiplier': cusum,
                'Real_Trades': trades,
                'Real_Win_Rate': win_rate
            })
            
    # 결과 정리
    res_df = pd.DataFrame(realistic_results)
    
    # 필터링 (10회 이상, 55% 이상)
    top_performers = res_df[
        (res_df['Real_Trades'] >= 10) & 
        (res_df['Real_Win_Rate'] >= 0.55)
    ].sort_values(by=['Real_Win_Rate', 'Real_Trades'], ascending=[False, False])
    
    print("\n" + "="*80)
    print("🏆 [현실 반영] 중복 제거 후 Top 20 전략 🏆")
    print("="*80)
    print(top_performers.head(20).to_markdown(index=False, floatfmt=".1%"))
    
    res_df.to_csv("realistic_strategy_ranking.csv", index=False)
    print("\n✅ 저장 완료: realistic_strategy_ranking.csv")

if __name__ == "__main__":
    analyze_realistic_performance()



>>> 현실적(Non-Overlapping) 성과 분석 중...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1071/1071 [00:56<00:00, 18.83it/s]



🏆 [현실 반영] 중복 제거 후 Top 20 전략 🏆
| strategy_name   | model    |   horizon_hours |   cusum_multiplier |   Real_Trades |   Real_Win_Rate |
|:----------------|:---------|----------------:|-------------------:|--------------:|----------------:|
| Sniper          | XGBoost  |              48 |             150.0% |            19 |           78.9% |
| Standard        | CatBoost |              24 |             150.0% |            13 |           76.9% |
| Sniper          | XGBoost  |              72 |             150.0% |            18 |           72.2% |
| Sniper          | XGBoost  |              96 |             150.0% |            18 |           72.2% |
| Trend           | CatBoost |             168 |             150.0% |            14 |           71.4% |
| Sniper          | XGBoost  |             168 |             150.0% |            16 |           68.8% |
| Sniper          | LightGBM |              96 |             100.0% |            19 |           68.4% |
| Sniper          | LightGBM |   

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. 파일 로드
OUTPUT_FOLDER = "../grid_search_result"
metrics_path = os.path.join(OUTPUT_FOLDER, "grid_search_metrics_FULL.csv")

if not os.path.exists(metrics_path):
    print("❌ 결과 파일이 없습니다. 먼저 Grid Search를 완료하세요.")
else:
    df = pd.read_csv(metrics_path)
    print(f"📂 데이터 로드 완료: {len(df)} rows")

    # =========================================================================
    # 🕵️‍♂️ [1단계] 필터링: "가짜 천재" 걸러내기
    # =========================================================================
    # 조건 1: Test F1 점수가 0.35 이상일 것 (최소한의 변별력)
    # 조건 2: Train과 Test의 F1 차이(Gap)가 0.2 이하일 것 (과적합 방지)
    # 조건 3: 정밀도(Precision)가 0.55 이상일 것 (동전 던지기보단 나아야 함)
    
    valid_strategies = df[
        (df['Test_F1'] >= 0.35) & 
        (df['Gap'] <= 0.20) &
        (df['Test_Prec'] >= 0.55)
    ].copy()

    print(f"\n🔍 [1차 필터링 결과] 생존한 전략 수: {len(valid_strategies)} / {len(df)}")
    
    if len(valid_strategies) > 0:
        # =====================================================================
        # 🏆 [2단계] 랭킹 산정: 누가 진짜 에이스인가?
        # =====================================================================
        # 종합 점수 = F1(40%) + Precision(40%) - Gap(20% 감점)
        valid_strategies['Score'] = (valid_strategies['Test_F1'] * 0.4) + \
                                    (valid_strategies['Test_Prec'] * 0.4) - \
                                    (valid_strategies['Gap'].abs() * 0.2)
        
        top_10 = valid_strategies.sort_values(by='Score', ascending=False).head(10)
        
        print("\n👑 [TOP 10 Strategies] (안정성 & 수익성 겸비)")
        print(top_10[['Strategy', 'Model', 'Horizon', 'CUSUM', 'Test_F1', 'Test_Prec', 'Gap']].to_string(index=False))
        
        # =====================================================================
        # 📊 [3단계] 인사이트: 어떤 설정이 좋았나?
        # =====================================================================
        print("\n📈 [Insight Analysis]")
        
        # 1. CUSUM n값의 영향
        best_cusum = valid_strategies.groupby('CUSUM')['Test_F1'].mean().sort_values(ascending=False)
        print(f"   - 최고의 CUSUM n값: {best_cusum.index[0]} (평균 F1: {best_cusum.iloc[0]:.3f})")
        
        # 2. 모델별 승률
        best_model = valid_strategies.groupby('Model')['Test_Prec'].mean().sort_values(ascending=False)
        print(f"   - 최고의 모델: {best_model.index[0]} (평균 정밀도: {best_model.iloc[0]:.3f})")
        
        # 3. 시간대별 성과
        best_horizon = valid_strategies.groupby('Horizon')['Test_F1'].mean().sort_values(ascending=False)
        print(f"   - 최고의 시간대(Horizon): {best_horizon.index[0]}시간")

    else:
        print("😢 조건에 맞는 전략이 하나도 없습니다. 파라미터나 데이터를 점검하세요.")


📂 데이터 로드 완료: 4284 rows

🔍 [1차 필터링 결과] 생존한 전략 수: 138 / 4284

👑 [TOP 10 Strategies] (안정성 & 수익성 겸비)
  Strategy    Model  Horizon  CUSUM  Test_F1  Test_Prec       Gap
QuickScalp  XGBoost      168    1.5 0.707692   0.576803  0.005791
QuickScalp  XGBoost       96    1.5 0.707692   0.576803  0.005791
QuickScalp  XGBoost       72    1.5 0.707692   0.576803  0.005791
QuickScalp  XGBoost       48    1.5 0.707692   0.576803  0.005791
QuickScalp CatBoost       72    2.0 0.718447   0.560606  0.000000
QuickScalp CatBoost       48    2.0 0.718447   0.560606  0.000000
QuickScalp CatBoost       96    2.0 0.718447   0.560606  0.000000
QuickScalp CatBoost      168    2.0 0.718447   0.560606  0.000000
QuickScalp  XGBoost       48    2.0 0.718447   0.560606 -0.003025
QuickScalp LightGBM       48    2.0 0.718447   0.560606 -0.003025

📈 [Insight Analysis]
   - 최고의 CUSUM n값: 2.0 (평균 F1: 0.670)
   - 최고의 모델: LightGBM (평균 정밀도: 0.568)
   - 최고의 시간대(Horizon): 168시간


In [13]:
import pandas as pd
import os

OUTPUT_FOLDER = "../grid_search_result"
logs_path = os.path.join(OUTPUT_FOLDER, "backend_deployment_logs_FIXED.csv")

print(f"📂 로그 로드 중: {logs_path}")
df = pd.read_csv(logs_path)

# 1. 특정 전략 & 시간대 필터링
target = df[
    (df['strategy'] == 'Balanced') & 
    (df['model'] == 'XGBoost') & 
    (df['cusum_multiplier'] == 0.5) &
    (df['horizon_hours'] == 96)
]

print(f"\n🔍 [Balanced / XGBoost / 0.5 / 96h] 샘플 분석")
print(f"   - 총 행 개수: {len(target)}")
print(f"   - 유니크한 시간(signal_time) 개수: {target['signal_time'].nunique()}")

# 2. 중복 여부 확인 사살
if len(target) > target['signal_time'].nunique():
    print("\n🚨 [경고] 중복 데이터 발견됨!")
    print(f"   - 중복된 행 수: {len(target) - target['signal_time'].nunique()}")
    
    # 중복된 시간 예시 출력
    dup_times = target['signal_time'].value_counts()
    print(f"   - 가장 많이 중복된 시간: {dup_times.index[0]} ({dup_times.iloc[0]}회 등장)")
    
    # 그 시간의 데이터 출력해보기
    dup_sample = target[target['signal_time'] == dup_times.index[0]]
    print("\n📝 [중복 샘플 데이터 확인]")
    print(dup_sample[['fold_id', 'ml_prediction', 'confidence']].to_string())
else:
    print("\n✅ 중복 없음. 데이터는 깨끗함. (모델이 그냥 많이 사는 놈임)")
    
    # 매수 비율 확인
    buy_ratio = target['ml_prediction'].mean()
    print(f"   - 매수 비율(Buy Ratio): {buy_ratio:.2f}")


📂 로그 로드 중: ../grid_search_result/backend_deployment_logs_FIXED.csv

🔍 [Balanced / XGBoost / 0.5 / 96h] 샘플 분석
   - 총 행 개수: 3260
   - 유니크한 시간(signal_time) 개수: 3260

✅ 중복 없음. 데이터는 깨끗함. (모델이 그냥 많이 사는 놈임)
   - 매수 비율(Buy Ratio): 0.74


In [15]:
import pandas as pd
import os

# 파일 로드
OUTPUT_FOLDER = "../grid_search_result"
logs_path = os.path.join(OUTPUT_FOLDER, "backend_deployment_logs_FIXED.csv")
df = pd.read_csv(logs_path)

print(f"📂 총 데이터: {len(df):,} rows")

# =============================================================================
# ★ 핵심: GroupBy로 '유니크한 신호' 단위로 묶어버리기
# =============================================================================
# 전략(Strat) + 모델(Model) + 설정(CUSUM, Horizon) + 시간(Time) 이 같으면
# -> 같은 신호다. (Fold가 달라도 같은 사건임)
# -> 따라서 mean()을 때려서 평균 예측값(확률)을 보거나, first()로 하나만 가져옴.
# -> 여기서는 '다수결'이나 '평균'이 아니라, 보수적으로 '첫 번째 기록' 사용.

unique_signals = df.groupby(
    ['strategy', 'model', 'cusum_multiplier', 'horizon_hours', 'signal_time']
).first().reset_index()

print(f"📉 유니크 신호 압축 완료: {len(unique_signals):,} rows")

# =============================================================================
# 📊 통계 산출 (이제 중복 걱정 없음)
# =============================================================================

stats = unique_signals.groupby(['strategy', 'model', 'cusum_multiplier', 'horizon_hours']).agg(
    Total_Count=('ml_prediction', 'count'),
    Buy_Count=('ml_prediction', 'sum') # ml_prediction은 0 또는 1
).reset_index()

# 승리 횟수: (매수함 1) AND (정답임 True) 인 경우
win_df = unique_signals[
    (unique_signals['ml_prediction'] == 1) & 
    (unique_signals['correct'] == True)
]
win_counts = win_df.groupby(['strategy', 'model', 'cusum_multiplier', 'horizon_hours']).size().reset_index(name='Win_Count')

# 병합
final = pd.merge(stats, win_counts, on=['strategy', 'model', 'cusum_multiplier', 'horizon_hours'], how='left').fillna(0)

# 지표 계산
final['Buy_Ratio'] = final['Buy_Count'] / final['Total_Count']
final['Win_Rate'] = final['Win_Count'] / final['Buy_Count']

# =============================================================================
# 🏆 랭킹 (조건: 10번 이상 매매, 매수비율 50% 미만)
# =============================================================================
ranking = final[
    (final['Buy_Count'] >= 10) &
    (final['Buy_Ratio'] < 0.50) &
    (final['Win_Rate'] >= 0.50)
].sort_values(by='Win_Rate', ascending=False).head(20)

print("\n✨ [최종] 중복 없는 깔끔한 랭킹 Top 20")
print(ranking[['strategy', 'model', 'cusum_multiplier', 'horizon_hours', 'Buy_Count', 'Win_Count', 'Win_Rate', 'Buy_Ratio']].to_string(index=False))
# ... (앞부분 로드 및 집계 코드 동일) ...

# =============================================================================
# 🎯 유효성 검증 필터 (Validation Filter)
# =============================================================================
# 기준 1: Buy_Count >= 100 (통계적 유의성 확보)
# 기준 2: Win_Rate >= 0.55 (수수료 떼고도 먹을 게 있어야 함)
# 기준 3: CUSUM Multiplier >= 1.0 (너무 민감한 0.5 제외)

validated_strats = final[
    (final['Buy_Count'] >= 100) & 
    (final['Win_Rate'] >= 0.55) &
    (final['cusum_multiplier'] >= 1.0)
].sort_values(by='Win_Rate', ascending=False).head(15)

print("\n🏆 [통계적으로 유효한] BEST STRATEGY (최소 100회 매매)")
print("="*100)
print(validated_strats[['strategy', 'model', 'cusum_multiplier', 'horizon_hours', 'Buy_Count', 'Win_Count', 'Win_Rate', 'Buy_Ratio']].to_string(index=False))
print("="*100)

if validated_strats.empty:
    print("😢 100회 이상 매매하면서 승률 55% 넘는 게 없습니다.")
    print("   -> 기준을 완화하세요 (Buy_Count >= 50 또는 Win_Rate >= 0.53)")


📂 총 데이터: 1,864,068 rows
📉 유니크 신호 압축 완료: 1,864,068 rows

✨ [최종] 중복 없는 깔끔한 랭킹 Top 20
strategy    model  cusum_multiplier  horizon_hours  Buy_Count  Win_Count  Win_Rate  Buy_Ratio
  Sniper  XGBoost               1.5             48         20       15.0  0.750000   0.013928
   Trend CatBoost               1.5            168         35       25.0  0.714286   0.024373
  Sniper LightGBM               1.0            168         19       13.0  0.684211   0.009048
  Sniper  XGBoost               1.5            168         19       13.0  0.684211   0.013231
  Sniper LightGBM               1.0             96         19       13.0  0.684211   0.009048
  Sniper  XGBoost               1.5             96         19       13.0  0.684211   0.013231
  Sniper  XGBoost               1.5             72         19       13.0  0.684211   0.013231
Standard CatBoost               1.5             24         15       10.0  0.666667   0.010446
Standard LightGBM               0.5             24         18       12.

In [25]:
import pandas as pd
import os

OUTPUT_FOLDER = "./"
metrics_path = os.path.join(OUTPUT_FOLDER, "grid_search_metrics_FULL.csv")

print(f"📂 Metrics 분석 중: {metrics_path}")
df = pd.read_csv(metrics_path)

# =============================================================================
# 🎯 [엄격한 필터] 과적합 박멸 (Overfitting Zero)
# =============================================================================
# 조건 1: Gap <= 0.15 (Train과 Test 차이가 15%p 이내)
# 조건 2: Test_F1 >= 0.35 (최소한의 기본 성능)
# 조건 3: Test_Prec >= 0.52 (찍기보단 잘해야 함)

clean_strats = df[
    (df['Gap'] <= 0.15) & 
    (df['Test_F1'] >= 0.35) &
    (df['Test_Prec'] >= 0.52)
].copy()

# =============================================================================
# 🏆 랭킹 산정
# =============================================================================
# 점수 = Test_F1 (성능) - Gap (안전성 보너스)
clean_strats['Score'] = clean_strats['Test_F1'] - clean_strats['Gap']

# Top 20 출력
top_clean = clean_strats.sort_values(by='Score', ascending=False).head(20)

print("\n🌿 [청정 구역] 과적합 없고(Stable), 성능 좋은(High F1) 전략 TOP 20")
print("="*100)
print(top_clean[['Strategy', 'Model', 'CUSUM', 'Horizon','Test_Accuracy', 'Test_F1', 'Train_F1', 'Gap', 'Test_Prec']].to_string(index=False))
print("="*100)

if top_clean.empty:
    print("😢 조건(Gap<0.15, F1>0.35)을 만족하는 '완벽한' 전략이 없습니다.")
    print("   -> 차선책: Gap < 0.20 으로 기준 완화 필요.")



📂 Metrics 분석 중: ./grid_search_metrics_FULL.csv

🌿 [청정 구역] 과적합 없고(Stable), 성능 좋은(High F1) 전략 TOP 20


KeyError: "['Test_Accuracy'] not in index"

In [34]:
import pandas as pd
import os

OUTPUT_FOLDER = "./"
metrics_path = os.path.join(OUTPUT_FOLDER, "grid_search_metrics_FULL.csv")
logs_path = os.path.join(OUTPUT_FOLDER, "backend_deployment_logs_FULL.csv")

print("📂 데이터 로드 및 병합 중...")

# 1. Metrics 로드 (과적합 정보)
df_met = pd.read_csv(metrics_path)
# 컬럼명 통일 (소문자)
df_met.columns = df_met.columns.str.lower().str.strip() 


# 2. Logs 로드 (승률/횟수 정보)
df_log = pd.read_csv(logs_path)
df_log.columns = df_log.columns.str.lower().str.strip()

# 2-1. 로그 집계 (유니크 신호 기준)
unique_signals = df_log.drop_duplicates(subset=['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours', 'signal_time'])

stats = unique_signals.groupby(['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours']).agg(
    Total_Count=('ml_prediction', 'count'),
    Buy_Count=('ml_prediction', 'sum')
).reset_index()

real_wins = unique_signals[
    (unique_signals['ml_prediction'] == 1) & 
    (unique_signals['is_correct'] == True)
].groupby(['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours']).size().reset_index(name='Win_Count')

df_stats = pd.merge(stats, real_wins, on=['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours'], how='left').fillna(0)
df_stats['Win_Rate'] = df_stats['Win_Count'] / df_stats['Buy_Count']
df_stats['Buy_Ratio'] = df_stats['Buy_Count'] / df_stats['Total_Count']



final_df = pd.merge(
    df_stats, 
    df_met, 
    left_on=['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours'],
    right_on=['strategy', 'model', 'cusum', 'horizon'], # 파일 컬럼명 확인 필요 (대소문자 등)
    how='inner'
)


candidates = final_df[
    (final_df['Win_Rate'] >= 0.53) &
    (final_df['Buy_Count'] >= 50) &
    (final_df['gap'] <= 0.15) & 
    (final_df['Buy_Ratio'] < 0.70)
].copy()

# 랭킹 점수 (승률 중요)
candidates['Final_Score'] = candidates['Win_Rate'] * 0.6 + (1 - candidates['gap']) * 0.4

top_candidates = candidates.sort_values(by='Final_Score', ascending=False).head(10)

print("\n💎 [삼위일체] 승률 + 횟수 + 안전성을 모두 갖춘 최종 후보")
print("="*120)
cols = ['strategy_name', 'model', 'cusum_multiplier', 'horizon_hours', 'Win_Rate', 'Buy_Count', 'gap', 'Buy_Ratio']
print(top_candidates[cols].to_string(index=False))
print("="*120)

if top_candidates.empty:
    print("😢 완벽한 후보가 없습니다. 기준을 조금씩 완화해보세요.")


📂 데이터 로드 및 병합 중...

💎 [삼위일체] 승률 + 횟수 + 안전성을 모두 갖춘 최종 후보
strategy_name    model  cusum_multiplier  horizon_hours  Win_Rate  Buy_Count      gap  Buy_Ratio
       Sniper LightGBM               1.5             48  0.636364         55 0.000000   0.040831
     Balanced  XGBoost               0.5              8  0.625000         56 0.000000   0.018337
     Balanced  XGBoost               0.5             24  0.580153        131 0.000000   0.042895
       Sniper LightGBM               1.5             48  0.636364         55 0.121893   0.040831
       Sniper LightGBM               1.5             24  0.639344         61 0.132567   0.045286
     Balanced LightGBM               1.5             12  0.546763        139 0.000000   0.103192
     Balanced  XGBoost               0.5             12  0.543210         81 0.000000   0.026523
     Balanced  XGBoost               1.5             12  0.542857        140 0.000000   0.103935
       Sniper LightGBM               1.5             24  0.639344      

In [43]:
import pandas as pd
import os

OUTPUT_FOLDER = "../grid_search_result"
metrics_path = os.path.join(OUTPUT_FOLDER, "grid_search_metrics_FULL.csv")
logs_path = os.path.join(OUTPUT_FOLDER, "backend_deployment_logs_FIXED.csv")

print("📂 데이터 로드 및 병합 중...")

# 1. Metrics 로드 (과적합 정보)
df_met = pd.read_csv(metrics_path)
# 컬럼명 통일 (소문자)
df_met.columns = df_met.columns.str.lower().str.strip() 
# 필요한 컬럼만 선택 (strategy, model, cusum, horizon, gap, test_f1)
# 주의: 파일 내 컬럼명이 'strategy', 'cusum', 'horizon' 인지 확인 필요
# (보통 'strategy', 'cusum', 'horizon' 등으로 저장됨)

# 2. Logs 로드 (승률/횟수 정보)
df_log = pd.read_csv(logs_path)
df_log.columns = df_log.columns.str.lower().str.strip()

# 2-1. 로그 집계 (유니크 신호 기준)
unique_signals = df_log.drop_duplicates(subset=['strategy', 'model', 'cusum_multiplier', 'horizon_hours', 'signal_time'])

stats = unique_signals.groupby(['strategy', 'model', 'cusum_multiplier', 'horizon_hours']).agg(
    Total_Count=('ml_prediction', 'count'),
    Buy_Count=('ml_prediction', 'sum')
).reset_index()

real_wins = unique_signals[
    (unique_signals['ml_prediction'] == 1) & 
    (unique_signals['correct'] == True)
].groupby(['strategy', 'model', 'cusum_multiplier', 'horizon_hours']).size().reset_index(name='Win_Count')

df_stats = pd.merge(stats, real_wins, on=['strategy', 'model', 'cusum_multiplier', 'horizon_hours'], how='left').fillna(0)
df_stats['Win_Rate'] = df_stats['Win_Count'] / df_stats['Buy_Count']
df_stats['Buy_Ratio'] = df_stats['Buy_Count'] / df_stats['Total_Count']

# 3. 조인 (Join) - Metrics + Stats
# 키: strategy_name(logs) <-> strategy(metrics)
# model <-> model
# cusum_multiplier <-> cusum
# horizon_hours <-> horizon

final_df = pd.merge(
    df_stats, 
    df_met, 
    left_on=['strategy', 'model', 'cusum_multiplier', 'horizon_hours'],
    right_on=['strategy', 'model', 'cusum', 'horizon'], # 파일 컬럼명 확인 필요 (대소문자 등)
    how='inner'
)

# =============================================================================
# 🎯 [Final Filter] 삼위일체 조건 적용
# =============================================================================
# 1. 승률: 53% 이상 (기준 살짝 완화해서 후보군 확보)
# 2. 횟수: 50회 이상 (최소한의 검증)
# 3. 과적합: Gap < 0.15 (안전 제일)
# 4. 매수 비중: 70% 미만 (무지성 매수 제외)

candidates = final_df[
    (final_df['Win_Rate'] >= 0.53) &
    (final_df['Buy_Count'] >= 50) &
    (final_df['gap'] <= 0.15) & 
    (final_df['Buy_Ratio'] < 0.70)
].copy()

# 랭킹 점수 (승률 중요)
candidates['Final_Score'] = candidates['Win_Rate'] * 0.6 + (1 - candidates['gap']) * 0.4

top_candidates = candidates.sort_values(by='Final_Score', ascending=False).head(10)

print("\n💎 [삼위일체] 승률 + 횟수 + 안전성을 모두 갖춘 최종 후보")
print("="*120)
cols = ['strategy', 'model', 'cusum_multiplier', 'horizon_hours', 'Win_Rate', 'Buy_Count', 'gap', 'Buy_Ratio']
print(top_candidates[cols].to_string(index=False))
print("="*120)

if top_candidates.empty:
    print("😢 완벽한 후보가 없습니다. 기준을 조금씩 완화해보세요.")


📂 데이터 로드 및 병합 중...

💎 [삼위일체] 승률 + 횟수 + 안전성을 모두 갖춘 최종 후보
  strategy    model  cusum_multiplier  horizon_hours  Win_Rate  Buy_Count       gap  Buy_Ratio
  Balanced  XGBoost               2.5              8  0.615385         52  0.000000   0.063107
QuickScalp CatBoost               0.5              8  0.559322        472 -0.052833   0.144785
  Balanced LightGBM               1.5             12  0.626374         91  0.066116   0.063370
  Standard  XGBoost               1.5             24  0.576923         52  0.000000   0.036212
  Balanced  XGBoost               2.5             12  0.562500         80  0.000000   0.097087
  Balanced  XGBoost               2.5             12  0.562500         80  0.000000   0.097087
  Balanced  XGBoost               0.5             12  0.561086        221  0.000000   0.067791
  Balanced LightGBM               2.5              8  0.560000         50  0.000000   0.060680
  Balanced LightGBM               1.5             12  0.626374         91  0.116443   0.0

In [45]:
import pandas as pd
import os

OUTPUT_FOLDER = "../grid_search_result"
logs_path = os.path.join(OUTPUT_FOLDER, "backend_deployment_logs_FIXED.csv")

print(f"📂 비교 분석 중: {logs_path}")
df = pd.read_csv(logs_path)
df.columns = df.columns.str.lower().str.strip()

# 1. 타겟 전략 설정 (Balanced / LightGBM / 1.5 / 12h)
# 컬럼명 매핑 주의 (strategy_name vs strategy 등)
strat_col = 'strategy_name' if 'strategy_name' in df.columns else 'strategy'

target_data = df[
    (df[strat_col] == 'Balanced') & 
    (df['model'] == 'LightGBM') & 
    (df['cusum_multiplier'] == 1.5) & 
    (df['horizon_hours'] == 12)
]

# 2. 유니크 신호 단위로 중복 제거 (CUSUM 이벤트 기준)
# signal_time이 같으면 같은 CUSUM 이벤트임.
unique_events = target_data.drop_duplicates(subset=['signal_time'])

print(f"\n🥊 [세기의 대결] 깡통 CUSUM vs AI 모델 (Balanced/LGBM/1.5/12h)")
print("="*80)

# -----------------------------------------------------------------------------
# A. 깡통 CUSUM 성적표 (ML 없이 무조건 매수했을 때)
# -----------------------------------------------------------------------------
total_cusum_events = len(unique_events)
# actual_direction: 실제 올랐으면 1, 내렸으면 0 (이게 CUSUM의 정답지)
cusum_wins = unique_events[unique_events['actual_direction'] == 1].shape[0]

cusum_win_rate = cusum_wins / total_cusum_events if total_cusum_events > 0 else 0

print(f"1️⃣  CUSUM 자체 성능 (Baseline)")
print(f"   - 총 신호 발생: {total_cusum_events}회")
print(f"   - 자연 승리 횟수: {cusum_wins}회")
print(f"   - 승률: {cusum_win_rate * 100:.2f}%")

# -----------------------------------------------------------------------------
# B. AI 모델 성적표 (ML이 거르고 걸러서 매수한 것)
# -----------------------------------------------------------------------------
# AI가 매수(1)라고 한 것만 추림
ai_buys = unique_events[unique_events['ml_prediction'] == 1]
total_ai_bets = len(ai_buys)
# 그 중에서 실제 오른 것 (Win)
ai_wins = ai_buys[ai_buys['actual_direction'] == 1].shape[0]

ai_win_rate = ai_wins / total_ai_bets if total_ai_bets > 0 else 0

print(f"\n2️⃣  AI 모델 성능 (With ML Filter)")
print(f"   - AI 매수 선택: {total_ai_bets}회 (거절율: {(1 - total_ai_bets/total_cusum_events)*100:.1f}%)")
print(f"   - AI 승리 횟수: {ai_wins}회")
print(f"   - 승률: {ai_win_rate * 100:.2f}%")

# -----------------------------------------------------------------------------
# 결론
# -----------------------------------------------------------------------------
diff = (ai_win_rate - cusum_win_rate) * 100
print("-" * 80)
print(f"📈 개선 효과 (Alpha): {diff:+.2f}%p")

if diff > 3.0:
    print("✅ 결론: 머신러닝이 '똥'을 아주 잘 피했습니다. 쓸모 있습니다!")
elif diff > 0:
    print("⚠️ 결론: 약간의 개선은 있지만, 드라마틱하진 않습니다.")
else:
    print("❌ 결론: AI가 헛똑똑이입니다. 그냥 CUSUM대로 사는 게 낫습니다.")
print("="*80)


📂 비교 분석 중: ../grid_search_result/backend_deployment_logs_FIXED.csv

🥊 [세기의 대결] 깡통 CUSUM vs AI 모델 (Balanced/LGBM/1.5/12h)
1️⃣  CUSUM 자체 성능 (Baseline)
   - 총 신호 발생: 1436회
   - 자연 승리 횟수: 604회
   - 승률: 42.06%

2️⃣  AI 모델 성능 (With ML Filter)
   - AI 매수 선택: 91회 (거절율: 93.7%)
   - AI 승리 횟수: 57회
   - 승률: 62.64%
--------------------------------------------------------------------------------
📈 개선 효과 (Alpha): +20.58%p
✅ 결론: 머신러닝이 '똥'을 아주 잘 피했습니다. 쓸모 있습니다!
